# Python setup

In [48]:
import subprocess 
import time
import glob
import os
from pathlib import Path
import shutil
import pandas as pd
import tempfile
from PIL import Image
import resource
from ast import literal_eval
import json

def run(obj,print_output=False):
    start = time.perf_counter()
    result = subprocess.run(obj["args"],capture_output=True,text=True)    
    if result.returncode != 0:
        print(result.stdout)
        print(result.stderr)
        result.check_returncode()
    else:
        if print_output:
            print(result.stdout)
        return { "delta": time.perf_counter() - start, "label": obj["label"],"output": result.stdout,"error": result.stderr, "return_code": result.returncode }
    
rid="linux-x64"
#converter_exe_rel="../../bin/release/net6.0/linux-x64/GraphLogicA"
converter_exe_rel=f"../../src/bin/release/net6.0/{rid}/GraphLogicA"
converter_exe = Path(converter_exe_rel).absolute().as_posix()
graphlogica_exe = converter_exe
minimizer_exe = shutil.which("ltsconvert")
voxlogica_exe = f"../../src/VoxLogicA_1.0-experimental_{rid}/VoxLogicA"
output="output"
#shutil.rmtree(output,ignore_errors=True)
os.makedirs(output,exist_ok=True)
images = glob.glob("test-images/*.png")
    
def mk_df(results,delta_label):
    return pd.DataFrame(results).set_index("label").rename(columns={"delta": delta_label}).drop(columns=["output","error","return_code"])
# !(cd ../.. && dotnet build -c release -r $rid)
resource.setrlimit(resource.RLIMIT_STACK, (resource.RLIM_INFINITY, resource.RLIM_INFINITY))

# Convert images

In [19]:
def converter(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix()], "label": label }
    #return (run(label,args))

converter_df = mk_df([ run(converter(image)) for image in images ],"conversionAndWrite")


# Minimize

In [49]:
def minimizer(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(
        Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    return {"args": [minimizer_exe, "--timings", "-ebranching-bisim", d_path.as_posix(), m_path.as_posix()], "label": label}


out = [run(minimizer(image)) for image in images]

def f(x: str):
    try:
        return float(x)
    except:
        return False

myLabels = { "reachability check": "reachabilityCheck", "total": "mcrl2-int" }

for o in out:
    lines: str = out[0]["error"].strip("- ").splitlines()
    # res = {
    #     x[0]: f(x[1]) for line in lines if (x := line.strip("' ").split(":")) if len(x) > 1 if f(x[1])
    # }
    for line in lines:
        x = line.strip("' ").split(":")
        if len(x) > 1:
            fl = f(x[1])
            l = x[0]
            if l in myLabels.keys():
                l = myLabels[l]
            if fl:
                o[l] = fl

minimizer_df = mk_df(out, "mcrl2")
minimizer_df


,mcrl2,reachabilityCheck,reduction,mcrl2-int
label,,,,
rai-1080,3.894210,0.199457,0.413722,3.886677
maze-128,0.036753,0.199457,0.413722,3.886677
maze-256,0.131859,0.199457,0.413722,3.886677
maze-512,0.507059,0.199457,0.413722,3.886677
maze-1024,2.032995,0.199457,0.413722,3.886677
maze-2048,8.091730,0.199457,0.413722,3.886677
maze-4096,32.987489,0.199457,0.413722,3.886677
maze-8192,133.980235,0.199457,0.413722,3.886677
rai-130,0.072062,0.199457,0.413722,3.886677


# Convert the minimized model back

In [50]:
def convertBack(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    j_path = o_path.joinpath(path.with_suffix(".json").name)
    return { "args": [converter_exe,"--convert",m_path.as_posix(),j_path.as_posix()], "label": label }
    #return (run(label,args))

backConverter_df = mk_df([ run(convertBack(image)) for image in images ],"convertBack")


# Convert without writing the file 

In [51]:
def fakeConverter(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".fake.aut").name
    d_path = o_path.joinpath(s_path)
    return { "args": [converter_exe,"--convert",path.as_posix(),d_path.as_posix(),"--fakeconversion"], "label": label }
    #return (run(label,args))

fakeConverter_df = mk_df([ run(fakeConverter(image)) for image in images ],"conversion")


# Model checking on images using VoxLogicA

In [52]:
def colour(r, g, b, is_graph=False):
    if is_graph:
        return f'''ap("{r:02x}{g:02x}{b:02x}")'''
    else:
        return f'''(red(img) =. {r}) & (green(img) =. {g}) & (blue(img) =. {b})'''


def save(basename, output,form, is_graph):
    p = Path(basename)
    n = p.with_suffix("").name
    if is_graph:
        return f'''save "{output}/{n}_{form}.json" {form}'''
    else:
        return f'''save "{output}/{n}_{form}.png" {form}'''


def mazeSpecification(path, is_graph):
    return f'''
            load img = "{path}"    
            let w = {colour(255,255,255,is_graph)}
            let b = {colour(0,0,255,is_graph)}
            let g = {colour(0,255,0,is_graph)}
            let form1 = through(N b,w) & through(N g,w)
            let form2 = through(N (w & !through(N g,w)),b) & through(N g,w)
            let form3 = through(N form1,b)
            {save(path,output,"form1",is_graph)}
            {save(path,output,"form2",is_graph)}
            {save(path,output,"form3",is_graph)}
        '''

def raiSpecification(path, is_graph):
    return f'''
            load img="{path}"
            
            let y = {colour(255,255,0,is_graph)}
            let c = {colour(0,255,255,is_graph)}
            let g = {colour(0,255,0,is_graph)}
            let m = {colour(255,0,255,is_graph)}
            let r = {colour(255,0,0,is_graph)}
            let b = {colour(0,0,255,is_graph)}
            let gr = {colour(191,191,191,is_graph)}
            let bl = {colour(0,0,0,is_graph)}
            let w = {colour(255,255,255,is_graph)}
            
            let Z(phi1,phi2) = through(N phi2,phi1)
            
            let form1 = y Z c Z g Z m Z r Z b Z gr Z bl Z w Z gr Z bl Z y
            {save(path,output,"form1",is_graph)}
        '''


def findSpec(img : str,is_graph = False):
    specs = [ ["maze",mazeSpecification],["rai",raiSpecification] ]
    for (prefix,spec) in specs:
        if Path(img).name.startswith(prefix):
            return spec(img,is_graph)
    return None

def modelChecker(image, spec, is_graph=False):
    path = Path(image)
    fname = tempfile.NamedTemporaryFile().name
    f = open(fname, "w")
    f.write(spec)
    f.close()
    if is_graph:
        exe = graphlogica_exe
    else:
        exe = voxlogica_exe
    return {"args": [exe, fname], "label": path.with_suffix("").name, "property": "maze"}

modelChecker_df = mk_df(
    [run(modelChecker(image,spec))
        for image in images if (spec:=findSpec(image)) if spec],
    "modelCheckingFull")


# Model Checking on the minimal graph using GraphLogicA

In [53]:
def graph(image):
    path = Path(image)
    o_path = Path(output)
    j_path = o_path.joinpath(path.with_suffix(".json").name)
    return(j_path)

modelCheckerMin_df = mk_df(
    [run(modelChecker(graph(image),spec,True))
        for image in images if (spec:=findSpec(graph(image),True)) if spec],
    "modelCheckingMin")

In [54]:
# Read automata statistics

def autSize(image):
    path = Path(image)
    label = path.with_suffix("").name
    o_path = Path(output)
    s_path = path.with_suffix(".aut").name
    d_path = o_path.joinpath(s_path)
    m_path = o_path.joinpath(Path(path.with_suffix("").name + "_min").with_suffix(".aut"))
    first_line = ""
    first_line_min = ""
    with open(d_path,"r") as file:
        first_line = file.readline().lstrip("des ")
    
    with open(m_path,"r") as file:
        first_line_min = file.readline().lstrip("des ")

    autSize = float(os.path.getsize(d_path)) / (1024 * 1024)
    minSize = float(os.path.getsize(m_path)) / (1024 * 1024)

    t = literal_eval(first_line)
    tmin = literal_eval(first_line_min)

    return { "transitions": t[1], "statesMin": tmin[2] , "transitionsMin": tmin[1], "label": label, "autSize": autSize, "minSize": minSize }

autSize_df = pd.DataFrame([ autSize(image) for image in images]).set_index("label")

    

# Gather image sizes and produce the final table

In [57]:
def size(imgpath):    
    path = Path(imgpath)
    img = Image.open(imgpath)
    imgSize = float(os.path.getsize(imgpath)) / (1024 * 1024)
    return { "pixels": img.width * img.height, "label": path.with_suffix("").name, "imgSize": imgSize}

size_df = pd.DataFrame([ size(image) for image in images]).set_index("label")

In [58]:
df = size_df.join(autSize_df).join(fakeConverter_df).join(converter_df).join(minimizer_df).join(backConverter_df).join(modelChecker_df).join(modelCheckerMin_df)
data = df.sort_values(by='pixels')
data

,pixels,imgSize,transitions,statesMin,transitionsMin,autSize,minSize,conversion,conversionAndWrite,mcrl2,reachabilityCheck,reduction,mcrl2-int,convertBack,modelCheckingFull,modelCheckingMin
label,,,,,,,,,,,,,,,,
maze-128,16384,0.000515,145924,7,21,2.471705,0.000319,0.380743,NaN,0.036753,0.199457,0.413722,3.886677,0.307818,0.421658,0.387913
rai-130,31200,0.001439,278584,155,899,4.827498,0.015048,0.333562,NaN,0.072062,0.199457,0.413722,3.886677,0.322233,0.418057,0.471373
maze-256,65536,0.001075,586756,7,21,10.354568,0.000319,0.324618,NaN,0.131859,0.199457,0.413722,3.886677,0.298812,0.447184,0.402223
rai-260,124800,0.003263,1118764,315,1841,20.272376,0.032214,0.347465,NaN,0.236640,0.199457,0.413722,3.886677,0.295371,0.512968,0.486361
maze-512,262144,0.003278,2353156,7,21,44.553496,0.000319,0.341032,NaN,0.507059,0.199457,0.413722,3.886677,0.286439,0.516263,0.401461
rai-540,518400,0.010072,4656604,460,2766,90.333901,0.048703,0.346932,NaN,1.067004,0.199457,0.413722,3.886677,0.305261,0.662231,0.509590
maze-1024,1048576,0.012036,9424900,7,21,184.340572,0.000319,0.396647,NaN,2.032995,0.199457,0.413722,3.886677,0.278271,0.585772,0.404434
rai-1080,2073600,0.028042,18644404,945,6965,384.283109,0.124261,0.758824,2.258849,3.894210,0.199457,0.413722,3.886677,0.687416,1.774195,0.548200
maze-2048,4194304,0.040990,37724164,7,21,793.733506,0.000319,0.468240,NaN,8.091730,0.199457,0.413722,3.886677,0.290939,0.915772,0.390601


# Export data

In [59]:
df.to_csv("table.csv")


For batch execution: 
    
    rm output -rf && jupyter nbconvert --to script --ExecutePreprocessor.timeout=1800 --execute testMin.ipynb